<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#CovidVaccine"](https://twitter.com/hashtag/CovidVaccine) from Twitter. 


In [1]:
# Write your code here

# (2)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"

driver = webdriver.Chrome(r"C:\Users\fatim\Desktop\chromedriver.exe")
driver.get(URL)

while True:
    try:
        driver.find_element_by_css_selector("button#load-more-trigger").click()
        WebDriverWait(driver,0.8).until(EC.invisibility_of_element_located((By.CSS_SELECTOR,".ipl-load-more__load-indicator")))
    except Exception:break

soup = BeautifulSoup(driver.page_source, 'lxml')        
movie_containers = soup.find_all(class_ = 'lister-item-content')
reviews = []

for container in movie_containers:
    text_review = container.find(class_ = 'content')
    review = text_review.get_text().lstrip()
    reviews.append(review)

d = {'review': reviews}
movie_reviews = pd.DataFrame(d)
reviews_100 = pd.DataFrame(movie_reviews.head(100))
print(reviews_100.head())

reviews_100.to_csv(r"C:\Users\fatim\Desktop\Output_CSV.csv", index=False)

                                              review
0  Every once in a while a movie comes, that trul...
1  This is a movie that only those who have felt ...
2  Truly a masterpiece, The Best Hollywood film o...
3  Joaquin Phoenix gives a tour de force performa...
4  Most of the time movies are anticipated like t...


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [2]:
# Write your code here

reviews_100 = pd.DataFrame(movie_reviews[0:100])

# (1) Removing Noise: Punctuations, Frequent words, Special/Rare words.

# Punctuation removal.
reviews_100['cleaned_review'] = reviews_100['review'].str.replace('[^\w\s]','')

# Frequent words removal.
high_freq = pd.Series(' '.join(reviews_100['cleaned_review']).split()).value_counts()[:10]
high_freq = list(high_freq.index)
reviews_100['cleaned_review'] = reviews_100['cleaned_review'].apply(lambda x: " ".join(x for x in x.split() if x not in high_freq))

# Rare words removal.
low_freq = pd.Series(' '.join(reviews_100['cleaned_review']).split()).value_counts()[-10:]
low_freq = list(low_freq.index)
reviews_100['cleaned_review'] = reviews_100['cleaned_review'].apply(lambda x: " ".join(x for x in x.split() if x not in low_freq))

# (2) Removing numbers.
reviews_100['cleaned_review'] = reviews_100['cleaned_review'].str.replace('\d+', '')

# (3) Removing stopwords.
from nltk.corpus import stopwords
import nltk
stop = stopwords.words('english')

reviews_100['cleaned_review'] = reviews_100['cleaned_review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# (4) Lower casing
reviews_100['cleaned_review'] = reviews_100['cleaned_review'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# (5) Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
reviews_100['cleaned_review'] = reviews_100['cleaned_review'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# (6) Lemmatization
from textblob import Word
reviews_100['cleaned_review'] = reviews_100['cleaned_review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

reviews_100.to_csv(r"C:\Users\fatim\Desktop\Output_CSV.csv", index=False)

reviews_100.head(5)

,review,cleaned_review
0,"Every once in a while a movie comes, that trul...",everi come truli make impact joaquin perform s...
1,This is a movie that only those who have felt ...,thi felt alon isol truli relat you understand ...
2,"Truly a masterpiece, The Best Hollywood film o...",truli masterpiec the best hollywood film one b...
3,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix give tour de forc perform fear...
4,Most of the time movies are anticipated like t...,most time movi anticip like end fall short way...


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [3]:
# Write your code here

# (1) Parts of Speech (POS) Tagging

from nltk import pos_tag
import nltk

reviews_100['POS_Tags'] = reviews_100['cleaned_review'].apply(lambda x: pos_tag(nltk.word_tokenize(x)))

def NounCounter(x):
    nouns = []
    for (word, pos) in x:
        if pos.startswith("NN"):
            nouns.append(word)
    return nouns

reviews_100["Nouns"] = reviews_100['POS_Tags'].apply(NounCounter)
reviews_100["Noun_Count"] = reviews_100["Nouns"].str.len()

def VerbCounter(x):
    verbs = []
    for (word, pos) in x:
        if pos.startswith("VB"):
            verbs.append(word)
    return verbs

reviews_100["Verbs"] = reviews_100['POS_Tags'].apply(VerbCounter)
reviews_100["Verb_Count"] = reviews_100["Verbs"].str.len()

def AdjectiveCounter(x):
    adjectives = []
    for (word, pos) in x:
        if pos.startswith("JJ"):
            adjectives.append(word)
    return adjectives

reviews_100["Adjectives"] = reviews_100['POS_Tags'].apply(AdjectiveCounter)
reviews_100["Adjective_Count"] = reviews_100["Adjectives"].str.len()

def AdverbCounter(x):
    adverbs = []
    for (word, pos) in x:
        if pos.startswith("RB"):
            adverbs.append(word)
    return adverbs

reviews_100["Adverbs"] = reviews_100['POS_Tags'].apply(AdverbCounter)
reviews_100["Adverb_Count"] = reviews_100["Adverbs"].str.len()

reviews_100 = reviews_100.drop(['Nouns', 'Verbs', 'Adjectives', 'Adverbs'], axis = 1) 

reviews_100.head(5)

,review,cleaned_review,POS_Tags,Noun_Count,Verb_Count,Adjective_Count,Adverb_Count
0,"Every once in a while a movie comes, that trul...",everi come truli make impact joaquin perform s...,"[(everi, JJ), (come, VBP), (truli, NNS), (make...",34,7,10,2
1,This is a movie that only those who have felt ...,thi felt alon isol truli relat you understand ...,"[(thi, NN), (felt, VBD), (alon, JJ), (isol, NN...",24,10,7,2
2,"Truly a masterpiece, The Best Hollywood film o...",truli masterpiec the best hollywood film one b...,"[(truli, NN), (masterpiec, VBZ), (the, DT), (b...",40,17,18,4
3,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix give tour de forc perform fear...,"[(joaquin, NN), (phoenix, NN), (give, VB), (to...",31,13,7,0
4,Most of the time movies are anticipated like t...,most time movi anticip like end fall short way...,"[(most, JJS), (time, NN), (movi, JJ), (anticip...",29,11,14,2


In [20]:
# (2) Constituency Parsing and Dependency Parsing

# Constituency Parsing

import os
java_path = r"C:\Program Files\Java\jre1.8.0_261\bin\java.exe"
os.environ['JAVAHOME'] = java_path

from nltk.parse.stanford import StanfordParser

con_parser = StanfordParser(path_to_jar=r"C:\Users\fatim\Desktop\stanford-parser-4.0.0\stanford-parser.jar",
                            path_to_models_jar=r"C:\Users\fatim\Desktop\stanford-parser-4.0.0\stanford-parser-4.0.0-models.jar")

reviews_100['50_words_only'] = reviews_100['cleaned_review'].apply(lambda x: " ".join(x.split()[0:50]))

constituency_parsing_trees = []

cleaned_reviews_list = reviews_100['50_words_only'].tolist()
for x in cleaned_reviews_list:
    con_parser_tree = list(con_parser.raw_parse(x))
    constituency_parsing_trees.append(con_parser_tree)

reviews_100['Constituency_Parsing_Tree'] = constituency_parsing_trees
print(reviews_100['Constituency_Parsing_Tree'].head(5))

# Print the Constituency Parsing Tree
con = con_parser.raw_parse(reviews_100['50_words_only'][0])
for line in con:
    line.draw()

<ipython-input-20-4d9e426b6fc3>:11: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  con_parser = StanfordParser(path_to_jar=r"C:\Users\fatim\Desktop\stanford-parser-4.0.0\stanford-parser.jar",


0    [[[(S\n  (ADVP (RB everi))\n  (VP\n    (VB com...
1    [[[(PRN\n  (S\n    (NP (NN thi))\n    (VP\n   ...
2    [[[(PRN\n  (S\n    (NP (NNS truli))\n    (VP\n...
3    [[[(S\n  (VP\n    (VB joaquin)\n    (S (NP (NN...
4    [[[(NP (JJS most) (NN time)), (NP\n  (NP (NN m...
Name: Constituency_Parsing_Tree, dtype: object


In [27]:
# Dependency Parsing

from nltk.parse.stanford import StanfordDependencyParser

dep_parser = StanfordDependencyParser(path_to_jar=r"C:\Users\fatim\Desktop\stanford-parser-4.0.0\stanford-parser.jar",
                            path_to_models_jar=r"C:\Users\fatim\Desktop\stanford-parser-4.0.0\stanford-parser-4.0.0-models.jar")

dependency_parsing_trees = []

for x in cleaned_reviews_list:
    dep_parser_tree = list(dep_parser.raw_parse(x))
    dependency_parsing_trees.append(dep_parser_tree)

reviews_100['Dependency_Parsing_Tree'] = dependency_parsing_trees
print(reviews_100['Dependency_Parsing_Tree'].head(5))

# Print the Dependency Parsing Tree
dep = list(dep_parser.raw_parse(reviews_100['50_words_only'][0]))
dep_tree = [parse.tree() for parse in dep][0]
print(dep_tree)

<ipython-input-27-4237c87190be>:5: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  dep_parser = StanfordDependencyParser(path_to_jar=r"C:\Users\fatim\Desktop\stanford-parser-4.0.0\stanford-parser.jar",


0    [defaultdict(<function DependencyGraph.__init_...
1    [defaultdict(<function DependencyGraph.__init_...
2    [defaultdict(<function DependencyGraph.__init_...
3    [defaultdict(<function DependencyGraph.__init_...
4    [defaultdict(<function DependencyGraph.__init_...
Name: Dependency_Parsing_Tree, dtype: object
(review
  (come everi (make truli (perform (joaquin impact))))
  (watch
    scenographi
    brillianc
    grotesqu
    haunt
    cringi
    hard
    watch
    time
    mesmer
    wont
    blink
    eye
    (funni tragic serious)
    (timethi moment emot rollercoast sometim multipl emot poppingup))
  (predict
    (actionriddl (typic far))
    (thrillerdrama superhero proper psycholog))
  singl
  (develop (charact best) (seen i ever))
  found
  help
  wa)


In [35]:
# (3) Named Entity Recognition

from nltk.tag import StanfordNERTagger

st = StanfordNERTagger(r"C:\Users\fatim\Desktop\stanford-ner-4.0.0\classifiers\english.muc.7class.distsim.crf.ser.gz",
                       r"C:\Users\fatim\Desktop\stanford-ner-4.0.0\stanford-ner.jar")

reviews_100['NER'] = reviews_100['cleaned_review'].apply(lambda x: st.tag(nltk.word_tokenize(x)))

def PersonNameCounter(x):
    person_names = []
    for (word, description) in x:
        if description.startswith("PERSON"):
            person_names.append(word)
    return person_names

reviews_100['Person_Names'] = reviews_100['NER'].apply(PersonNameCounter)
reviews_100['Person_Names_Count'] = reviews_100['Person_Names'].str.len()

def OrganizationCounter(x):
    organizations = []
    for (word, description) in x:
        if description.startswith("ORGANIZATION"):
            organizations.append(word)
    return organizations

reviews_100['Organizations'] = reviews_100['NER'].apply(OrganizationCounter)
reviews_100['Organizations_Count'] = reviews_100['Organizations'].str.len()

def LocationCounter(x):
    locations = []
    for (word, description) in x:
        if description.startswith("LOCATION"):
            locations.append(word)
    return locations

reviews_100['Locations'] = reviews_100['NER'].apply(LocationCounter)
reviews_100['Locations_Count'] = reviews_100['Locations'].str.len()

def ProductCounter(x):
    products = []
    for (word, description) in x:
        if description.startswith("PRODUCT"):
            products.append(word)
    return products

reviews_100['Products'] = reviews_100['NER'].apply(ProductCounter)
reviews_100['Products_Count'] = reviews_100['Products'].str.len()

def DateCounter(x):
    dates = []
    for (word, description) in x:
        if description.startswith("DATE"):
            dates.append(word)
    return dates

reviews_100['Dates'] = reviews_100['NER'].apply(DateCounter)
reviews_100['Dates_Count'] = reviews_100['Dates'].str.len()

reviews_100 = reviews_100.drop(['Person_Names', 'Organizations', 'Locations', 'Products', 'Dates'], axis = 1) 
reviews_100[['cleaned_review', 'Person_Names_Count', 'Organizations_Count', 'Locations_Count', 'Products_Count', 'Dates_Count']].head(5)

,cleaned_review,Person_Names_Count,Organizations_Count,Locations_Count,Products_Count,Dates_Count
0,everi come truli make impact joaquin perform s...,0,0,0,0,0
1,thi felt alon isol truli relat you understand ...,0,0,0,0,0
2,truli masterpiec the best hollywood film one b...,0,0,0,0,0
3,joaquin phoenix give tour de forc perform fear...,0,0,0,0,0
4,most time movi anticip like end fall short way...,0,0,0,0,1


**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [1]:
'''
Write your explanations of the constituency parsing tree and dependency parsing tree here

Constituency Parsing Tree:

I have created the Constituency Parsing Tree for the first 50 words in each review. This is because the parser uses a lot of memory. 

I have printed out the Constituency Parsing Tree for the first review in our reviews_100 table. 
The Tree prints out in a new NLTK window. This tree divides a sentence into its constituents. 
Words are the constituents of a sentence and make up the terminal nodes of a parsing tree. 

At the first level below the ROOT, the review is fragmented into 4S, 1 FRAG, 1 Adjective Phrase (ADJP) and 1 Noun Phrase (NP).

1. The first S is then split into an Adverb Phrase (ADVP) and a Verb Phrase (VP).
   The ADVP has 1 Adverb (RB) 'everi'.
   The VP has 1 Verb (VB) 'come' and 1 S.
       The S has 1 NP and 1 VP.
       The NP has 1 Plural Noun (NNS) 'truli'
       The VP has 1 VB 'make' and 1 S.
           The S has 1 NP and 1 VP.
           The NP has 2 Nouns (NN) 'imapct' and 'joaquin'.
           The VP has 1 VB 'perform'

2. The FRAG is split into 3 NPs.
   The first NP has 13 NNs.
   The second NP has 2 Adjectives (JJ) and NNS.
   The third NP has 8 NNs.
   
3. The second S is split into 1 NP and 1 VP.
   The NP has 1 ADJP and 1 NN 'actionriddl'.
       The ADJP has 1 RB 'far' and 1 JJ 'typic'.
    The VP has 1 VBP 'predict' and 1 NP.
       The NP has 2 JJs, 1 NN and 1 NNS.
       
4. The ADJP has 1 JJ 'singl'.

5. The third S has 1 NP and 1 VP.
   The NP has 1 JJS 'best' and 1 NN 'charact'
   The VP has 1 VBP 'develop' and 1 S.
       The S has 1 NP, 1 ADVP and 1 VP.
       The NP has 1 PRP 'i'.
       The ADVP has 1 RB 'ever'.
       The P has 1 VBN 'seen'.
       
6. The fourth S has 1 VP.
   The VP has 1 VBN 'found'.
   
7. The NP has 3 NNs. 

******************************************************************************************************************************

Dependency Parsing Tree:

I have created the Dependency Parsing Tree for the first 50 words in each review. This is because the parser uses a lot of memory. 

I have printed out the Dependency Parsing Tree for the first review in our reviews_100 table. 
The Dependency Parsing Tree shows the syntax of a sentence by highlighting dependencies between the words.

The roots of the tree are 'come', 'watch', 'predict', 'singl', 'develop', 'found', 'help' and 'wa'.

1.'come' node has 'everi' and 'make' depending on it.
    Furthermore, 'make' has 'truli' and 'perform' depending on it.
        Then 'perform' has 'joaquin' depending on it.
            And, lastly, 'joaquin' has 'impact' depending on it.
            
2.'watch' node has 14 words depending on it: 'scenographi', 'brillianc', 'grotesqu', ..., 'funni' and 'timethi'.
    12 of the words do not have any further words depending on them.
    'funni' has 'tragic' and 'serious' depending on it.
    'timethi' has 7 words depending on it: 'moment', 'emot', 'rollercoast', etc.
    
3.'predict' node has 'actionriddl' and 'thrillerdrama' depending on it. 
    Furthermore, 'actionriddl' has 'typic' depending on it.
        And, lastly, 'typic' has 'far' depending on it.
    Furthermore, 'thrillerdrama' has 'superhero', 'proper' and 'psycholog' depending on it.

4.'develop' has 'charact' and 'seen' depending on it.
    Furthermore, 'charact' has 'best' depending on it.
    And, 'seen' has 'i' and 'ever' depending on it.
    
5. 'singl', 'found', 'help' and 'wa' are root words with no other words depending on them. 

'''

'\nWrite your explanations of the constituency parsing tree and dependency parsing tree here\n\n\n\n'